In [1]:
import cv2
import numpy as np
import os
import pandas as pd
from collections import Counter

In [2]:
# global variable
path = 'C:\Users\VE-ENG\Desktop\yale/'

In [3]:
def loadTrainImageSet(add):
    trainingImages = []
    trainingIndices = []
    for i in os.listdir(add):
        if 'normal' in i:
            try:
                img = cv2.imread(add+i,cv2.IMREAD_GRAYSCALE)
                trainingImages.append(np.asarray(img))
                number = filter(str.isdigit, i)
                trainingIndices.append(int(number))             
            except:
                print 'load %s failed'%i
    return trainingImages, trainingIndices

In [4]:
def loadTestImageSet(add):
    testingImages = []
    testingIndices = []
    for i in os.listdir(add):
        if 'normal' not in i:
            try:
                img = cv2.imread(add+i,cv2.IMREAD_GRAYSCALE)
                testingImages.append(np.asarray(img))
                number = filter(str.isdigit, i)
                testingIndices.append(int(number))             
            except:
                print 'load %s failed'%i
    return testingImages, testingIndices

In [5]:
def fit_predict_Recognizer(trainingImages,trainingIndices,testingImages,testingIndices,recognizerType = 1):
    # select recognizer
    if recognizerType == 0:
        recognizer = cv2.createEigenFaceRecognizer()
        print ('Eigen Face Recognizer selected')
    elif recognizerType == 1:
        recognizer = cv2.createLBPHFaceRecognizer()
        print ('LBPH Face Recognizer selected')
    elif recognizerType == 2:
        recognizer = cv2.createFisherFaceRecognizer()
        print ('Fisher Face Recognizer selected')

    # train recognizer
    recognizer.train(np.asarray(trainingImages, dtype=np.uint8), np.asarray(trainingIndices, dtype=np.int))
    print ('Trained FaceRecognizer')
    
    
    characteristic = ['centerlight','glasses','happy','leftlight','noglasses','rightlight','sad','sleepy','surprised','wink']
    predictionList = []
    confidenceList = []

    accuracyList = []
    
    # testing the recognizer
    for c in characteristic:
        count = 0
        for i in np.unique(testingIndices):
            loadname = 'C:/Users/VE-ENG/Desktop/yale/subject'+str(i).zfill(2)+'_'+c+'.bmp'
            img = cv2.imread(loadname,cv2.IMREAD_GRAYSCALE)
            prediction, confidence = recognizer.predict(img)
            predictionList.append(int(prediction))
            confidenceList.append(confidence)           
            if prediction == i:
                count += 1
        
        accuracy = float(count)/len(np.unique(testingIndices))
#         print 'accuracy of %s is %f'%(c, accuracy) 
        accuracyList.append(accuracy)
        
#     print 'overall accuracy is %f' % (sum(accuracyList)/len(accuracyList))
    return predictionList, testingIndices, confidenceList, accuracyList

In [6]:
trainingImages, trainingIndices = loadTrainImageSet(path)
testingImages, testingIndices = loadTestImageSet(path)

# print testingIndices

predictionList = (np.zeros((len(testingIndices),3)))
accuracyList = (np.zeros((10,3)))
for i in range(3):
    prediction, actual, confidence, accuracy = fit_predict_Recognizer(trainingImages,trainingIndices,\
                                                testingImages,testingIndices,recognizerType = i)
    predictionList[:,i] = np.array(prediction).flatten()
    accuracyList[:,i] = np.array(accuracy).flatten()
    print ('-----------------------------------')
    
characteristic = ['centerlight','glasses','happy','leftlight','noglasses','rightlight','sad','sleepy','surprised','wink']    
df = pd.DataFrame(accuracyList, characteristic, columns=['Eigen', 'LBPH', 'Fisher'])
df

Eigen Face Recognizer selected
Trained FaceRecognizer
-----------------------------------
LBPH Face Recognizer selected
Trained FaceRecognizer
-----------------------------------
Fisher Face Recognizer selected
Trained FaceRecognizer
-----------------------------------


,Eigen,LBPH,Fisher
centerlight,0.400000,0.733333,0.333333
glasses,0.933333,1.000000,0.866667
happy,0.933333,0.933333,0.866667
leftlight,0.200000,0.666667,0.266667
noglasses,1.000000,1.000000,0.933333
rightlight,0.266667,0.600000,0.266667
sad,0.933333,1.000000,0.933333
sleepy,0.933333,0.933333,0.933333
surprised,0.733333,0.933333,0.666667
wink,0.933333,0.866667,0.933333


In [7]:
df.describe()

,Eigen,LBPH,Fisher
count,10.000000,10.000000,10.000000
mean,0.726667,0.866667,0.700000
std,0.313404,0.147406,0.295230
min,0.200000,0.600000,0.266667
25%,0.483333,0.766667,0.416667
50%,0.933333,0.933333,0.866667
75%,0.933333,0.983333,0.933333
max,1.000000,1.000000,0.933333


In [8]:
for i in range(3):
    accuracyWithoutLigh =[]
    for c,a in zip(characteristic, accuracyList[:,i]):
        if 'light' not in c:
            accuracyWithoutLigh.append(a)
    print "Model %d accuracy (without light affect): %f" % (i , sum(accuracyWithoutLigh)/len(accuracyWithoutLigh))

Model 0 accuracy (without light affect): 0.914286
Model 1 accuracy (without light affect): 0.952381
Model 2 accuracy (without light affect): 0.876190


In [9]:
def voting(p1, p2, p3, trust = 0):
#     trust : when all predicitons are different, choose the most trust one as result
    count = Counter([p1,p2,p3])
    maxx = max(count.values())
    if maxx != 1:
        keys = [x for x,y in count.items() if y ==maxx]
        return keys[0]
    else:
        return count.keys()[trust]

In [10]:
trust = 0

recognizer1 = cv2.createEigenFaceRecognizer()
recognizer2 = cv2.createLBPHFaceRecognizer()
recognizer3 = cv2.createFisherFaceRecognizer()

#     train recognizer
recognizer1.train(np.asarray(trainingImages, dtype=np.uint8), np.asarray(trainingIndices, dtype=np.int))
recognizer2.train(np.asarray(trainingImages, dtype=np.uint8), np.asarray(trainingIndices, dtype=np.int))
recognizer3.train(np.asarray(trainingImages, dtype=np.uint8), np.asarray(trainingIndices, dtype=np.int))    
    
characteristic = ['centerlight','glasses','happy','leftlight','noglasses','rightlight','sad','sleepy','surprised','wink']
result = []    
count = 0
#     testing the recognizer
for c in characteristic:
    for i in np.unique(testingIndices):
        loadname = 'C:/Users/VE-ENG/Desktop/yale/subject'+str(i).zfill(2)+'_'+c+'.bmp'
        img = cv2.imread(loadname,cv2.IMREAD_GRAYSCALE)
        prediction1,confidence1= recognizer1.predict(np.asarray(img))
        prediction2,confidence2= recognizer2.predict(np.asarray(img))
        prediction3,confidence3= recognizer3.predict(np.asarray(img))
        
        finalPredition = voting(prediction1, prediction2, prediction3, trust)
        if finalPredition == i:
            count += 1
        result.append([str(i).zfill(2)+'_'+c, i, prediction1, prediction2, prediction3, finalPredition])

print count
print float(count)/len(result)
df = pd.DataFrame(result, columns =["File Name", 'Actual', 'EigenFace', 'LBPH','Fisher', 'Final'])
df

111
0.74


,File Name,Actual,EigenFace,LBPH,Fisher,Final
0,01_centerlight,1,1,10,2,1
1,02_centerlight,2,15,2,1,1
2,03_centerlight,3,14,3,14,14
3,04_centerlight,4,4,4,4,4
4,05_centerlight,5,5,5,5,5
5,06_centerlight,6,1,3,1,1
6,07_centerlight,7,6,7,6,6
7,08_centerlight,8,5,3,5,5
8,09_centerlight,9,15,9,15,15
9,10_centerlight,10,14,10,14,14


In [11]:
throushould = 75
recognizer = cv2.createLBPHFaceRecognizer()
recognizer.train(np.asarray(trainingImages, dtype=np.uint8), np.asarray(trainingIndices, dtype=np.int))
result = []
for i in range(15):
    loadname = 'C:/Users/VE-ENG/Desktop/yale/subject'+str(i+1).zfill(2)+'_centerlight.bmp'
    img = cv2.imread(loadname,0)
    prediction,confidence= recognizer.predict(np.asarray(img))
    x = 'no' if confidence > throushould else 'yes'
    result.append([i+1, prediction, confidence, x])
    
data = pd.DataFrame(result, columns = ['Act.', 'Pred.', 'Confidence', 'Judge'])
print data['Confidence'].min(axis=0)
print data['Confidence'].max(axis=0)
print data['Confidence'].mean(axis=0)
print data
# confidence = distance btw target and trained image (lower is better)

54.918825737
110.494530001
84.6521711212
    Act.  Pred.  Confidence Judge
0      1     10  110.494530    no
1      2      2   87.204482    no
2      3      3   86.245827    no
3      4      4   91.992137    no
4      5      5   94.365282    no
5      6      3  103.547384    no
6      7      7   54.918826   yes
7      8      3   92.756788    no
8      9      9   92.601554    no
9     10     10   69.050291   yes
10    11     11   74.329479   yes
11    12     12   82.342551    no
12    13     13   68.930056   yes
13    14      3   83.493798    no
14    15     15   77.509581    no


In [12]:
# loadname = 'C:/Users/VE-ENG/Desktop/0_1.bmp'
loadname = 'C:/Users/VE-ENG/Desktop/yale/subject10_wink.bmp'
img = cv2.imread(loadname,0)
prediction,confidence= recognizer.predict(np.asarray(img))
print prediction, confidence

10 42.8204048365
